# 🏠 Smart Intruder Detection for Property Surveillance using VideoDB RTStream
[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/video-db/videodb-cookbook/blob/main/real_time_streaming/Intrusion_Detection.ipynb)
## 📖 Storytime: Why This Matters

Have you ever felt anxious leaving your home, shop, or property unattended?  
**What if someone’s lurking around your property?**  
**What if someone’s trying the door or peeking through windows?**  
Sure — you could install IP cameras, but who has the time to watch them 24/7?

**Good news — you don’t have to anymore.**

With **VideoDB RTStream**, you can build a smart, AI-powered property surveillance system that actively monitors live video streams, detects suspicious activity, and immediately sends alerts for escalating security breaches — all without human supervision.

---

## 🚀 What You’ll Build in This Notebook

In this notebook, we’ll build a layered intruder detection system with **three levels of security breach detection**:
- **🔸 Level 1: Loitering near the property perimeter**
- **🔸 Level 2: Attempting to check the door lock (tampering / intrusion attempt)**
- **🔸 Level 3: Physically entering the property (highest breach)**

You’ll learn how to:
- 📺 Connect a property surveillance video stream to VideoDB
- 🤖 Use AI to continuously analyze scenes for suspicious activity
- Detect and differentiate **three levels of security threats**
- 🚨 Trigger separate real-time alerts for each level to a webhook

Let’s build it!

---

## 📦 Step 1: Install Dependencies

We’ll begin by installing the VideoDB SDK

In [1]:
!pip install -q videodb

  Preparing metadata (setup.py) ... done


---
## 📦 Step 2: Connect to VideoDB

Let's connect to VideoDB's API using your credentials to prepare for stream monitoring.

Please enter your `VIDEO_DB_API_KEY` in the input box that appears below after you run this cell.

Your input will be masked.


In [2]:
import videodb
import os
from getpass import getpass

api_key = getpass("Please enter your VideoDB API Key: ")

os.environ["VIDEO_DB_API_KEY"] = api_key

conn = videodb.connect()
coll = conn.get_collection()

print("Connected to VideoDB securely!")

Please enter your VideoDB API Key: ··········
Connected to VideoDB securely!


---

## 📦 Step 3: Connect to the Property Surveillance RTSP Stream

We’ll connect to the live video stream of your property’s security camera.

In this demo, the stream is running at `rtsp://samples.rts.videodb.io:8554/intruder`.


In [ ]:
rtsp_url = "rtsp://samples.rts.videodb.io:8554/intruder"
intruder_stream = coll.connect_rtstream(
    name="Property Security Stream",
    rtsp_url=rtsp_url,
)
print(intruder_stream)

#### Let us list all the rtstreams in our collection.

In [ ]:
def list_rtstreams():
    for rtstream in coll.list_rtstreams():
        print(f"""RTStream:
        ID            : {rtstream.id}
        Name          : {rtstream.name}
        Collection ID : {rtstream.collection_id}
        Created At    : {rtstream.created_at}
        Sample Rate   : {rtstream.sample_rate}
        Status        : {rtstream.status}
        """)
        print("-" * 80)

list_rtstreams()

RTStream:
        ID            : rts-019711db-1086-7750-ba79-8f47a4fed603
        Name          : Cricket Finals Stream
        Collection ID : c-81fc6459-fe30-44ac-8c5b-ea0898c2e152
        Created At    : 2025-05-27T13:07:38.758741
        Sample Rate   : 30
        Status        : stopped
        
--------------------------------------------------------------------------------
RTStream:
        ID            : rts-019711a0-0fde-7911-b282-25bc0b4ecf65
        Name          : Baby Crib Monitor
        Collection ID : c-81fc6459-fe30-44ac-8c5b-ea0898c2e152
        Created At    : 2025-05-27T12:03:11.966358
        Sample Rate   : 30
        Status        : stopped
        
--------------------------------------------------------------------------------
RTStream:
        ID            : rts-019710fa-9511-79c3-a924-e229e4815410
        Name          : Property Security Stream
        Collection ID : c-81fc6459-fe30-44ac-8c5b-ea0898c2e152
        Created At    : 2025-05-27T09:02:27.08927


#### If you have already connected the stream, run the below cell with the **rtstream id** to reconnect.

In [3]:
# intruder_stream = coll.get_rtstream("")

In [ ]:
# To stop the stream
# intruder_stream.stop()

In [4]:
# To start the stream
# intruder_stream.start()

---
### 👀 Let us have a look at the camera feed

#### 📺 Helper Functions: Search and Display

This cell contains all the utility functions to search, fetch, and visualize video streams. You don't need to modify this code.

In [ ]:
# To display the stream with relevant information

from IPython.display import HTML
import re
import time
from datetime import datetime, UTC
from videodb import play_stream


def display_stream(video_url, video_name="🎥 Camera Feed"):
    match = re.search(r"/(\d{16})-(\d{16})\.m3u8", video_url)
    if match:
        start_ts = int(match.group(1)) / 1e6
        end_ts = int(match.group(2)) / 1e6
        start_time = datetime.fromtimestamp(start_ts, UTC).strftime("%Y-%m-%d %H:%M:%S")
        end_time = datetime.fromtimestamp(end_ts, UTC).strftime("%Y-%m-%d %H:%M:%S")
        time_range = f"{start_time} → {end_time} UTC"
    else:
        time_range = "Time Unknown"

    video_player_html = play_stream(video_url)

    return HTML(f"""
    <div style="position:relative;width:640px;">
      {video_player_html._repr_html_() if hasattr(video_player_html, "_repr_html_") else video_player_html}
      <div style="position:absolute;top:10px;left:10px;background:rgba(0,0,0,0.6);color:#fff;padding:6px 12px;border-radius:4px;font-size:13px;font-family:sans-serif;">
        <strong>{video_name}</strong><br>{time_range}
      </div>
    </div>
    """)


# To dynamically set the display duration


def prompt_to_time(prompt):
    now = int(time.time())
    prompt = (
        f"It's {now} in epoch seconds. "
        f"Convert the phrase '{prompt}' into JSON "
        f'with keys "from" and "to" (both epoch seconds)'
    )

    result = coll.generate_text(
        prompt=prompt,
        model_name="pro",
        response_type="json",
    )
    output = result.get("output", {})
    return output.get("from"), output.get("to")


# To fetch stream


def fetch_stream(rtstream):
    _from, to = prompt_to_time("Show me last 5 mins")
    stream_url = rtstream.generate_stream(_from, to)
    return stream_url


#### 🔗 Get & Display Recent Stream

This cell uses the helper functions above to fetch and display the last few minutes of the stream.

In [ ]:
# To get last few minutes stream link

video_url = fetch_stream(intruder_stream)

video_name = "🚨 Private Property · Intruder Detection"
display_stream(video_url, video_name)

---

## 📦 Step 4: Index Scenes and Detect Suspicious Activity

We’ll create a real-time scene index that periodically analyzes video frames and generates natural language descriptions of what’s happening on the property.

The AI will look for:
- People loitering
- Approaching or physically interacting with the door/lock
- Attempting to enter the property

In [ ]:
intruder_scene_index = intruder_stream.index_visuals(
    batch_config={
        "type": "time",
        "value": 5,
        "frame_count": 2,
    },
    prompt="Monitor the area around the house closely. Detect and classify human presence around the house as either loitering, interacting with the door/lock, or entering the house; otherwise, consider the area safe.",
    name="Intruder_Detection_Index",
)
intruder_index_id = intruder_scene_index.rtstream_index_id
print("Scene Index ID:", intruder_index_id)

#### Let us list the scene indexes created on our rtstream.

In [ ]:
def list_rtstream_indexes(rtstream):
    # List live stream indexes
    rtstream_indexes = rtstream.list_scene_indexes()
    for rtstream_index in rtstream_indexes:

        print(f"""RTStreamSceneIndex:
            Index ID       : {rtstream_index.rtstream_index_id}
            RTStream ID    : {rtstream_index.rtstream_id}
            Name           : {rtstream_index.name}
            Status         : {rtstream_index.status}
            Config         : {rtstream_index.extraction_config}
            Prompt         : {rtstream_index.prompt}
        """)
        print("-" * 80)

list_rtstream_indexes(intruder_stream)

RTStreamSceneIndex:
            Index ID       : 7a6b9e4dc0132ebe
            RTStream ID    : rts-019710fa-9511-79c3-a924-e229e4815410
            Name           : Intruder_Detection_Index
            Status         : running
            Config         : {'frame_count': '2', 'time': '5'}
            Prompt         : Monitor the area around the house closely. Detect and classify human presence around the house as either loitering, interacting with the door/lock, or entering the house; otherwise, consider the area safe.
        
--------------------------------------------------------------------------------
RTStreamSceneIndex:
            Index ID       : caf71652aed1141c
            RTStream ID    : rts-019710fa-9511-79c3-a924-e229e4815410
            Name           : Intruder_Security_Index
            Status         : stopped
            Config         : {'frame_count': '3', 'time': '5'}
            Prompt         : Monitor the area around the house closely. Detect and classify huma


#### If you have already created a scene index, run the below cell with your **scene index id** to reconnect.

In [ ]:
# intruder_index_id = ""
# intruder_scene_index = intruder_stream.get_scene_index(intruder_index_id)

In [ ]:
# To stop the index
# intruder_scene_index.stop()

In [ ]:
# To start the index
# intruder_scene_index.start()

---
### Let us see the generated indexes

In [ ]:
import time
from datetime import datetime
from zoneinfo import ZoneInfo

def _convert_to_ist(timestamp: float) -> str:
    """Convert UTC timestamp to IST (Asia/Kolkata) datetime string."""
    return (
        datetime.fromtimestamp(timestamp)
        .astimezone(ZoneInfo("Asia/Kolkata"))
        .strftime("%Y-%m-%d %H:%M:%S")
    )

def get_scenes(index):
    # Print indexed scenes
    scenes = index.get_scenes(page_size=5)
    if scenes:
        for scene in scenes.get("scenes"):
            start = _convert_to_ist(scene["start"])
            end = _convert_to_ist(scene["end"])
            description = scene["description"]
            print(f"{start}-{end}: {description}")
            print("-" * 80)
    else:
        print("Scenes not found for given index.")

get_scenes(intruder_scene_index)

2025-05-27 20:25:06-2025-05-27 20:25:10 : Based on the image, I can see a person near the left side of the image. The person is near a lawn mower. It is difficult to determine the person's intent or activity with certainty. The person is not interacting with the door/lock or entering the house. Therefore, the person is loitering.
--------------------------------------------------------------------------------
2025-05-27 20:24:46-2025-05-27 20:24:52 : Based on the image, there is a human presence loitering in the distance.
--------------------------------------------------------------------------------
2025-05-27 20:24:39-2025-05-27 20:24:44 : Here's a breakdown of the situation based on the images provided:

**Analysis:**

*   **Human Presence Detected:** Yes, there are at least two individuals visible in the images. One is near the left side of the image, and another is in the background.
*   **Classification:**
    *   The individual on the left appears to be walking or loitering.
  

---
## 🔌 WebSocket Connection (Optional)

The `index_visuals()` method supports an optional `ws_connection_id` parameter for receiving real-time events through WebSocket connections instead of webhooks. This provides a persistent, bidirectional connection for instant event delivery.



In [ ]:
# # Optional: Use WebSocket for real-time events
# import asyncio
# 
# # Initialize WebSocket connection first
# ws = conn.connect_websocket()
# await ws.connect()
# connection_id = ws.connection_id
#
# scene_index = intruder_stream.index_visuals(
#     batch_config={"type": "time", "value": 5, "frame_count": 2},
#     prompt="Monitor the area around the house closely. Detect and classify human presence around the house as either loitering, interacting with the door/lock, or entering the house; otherwise, consider the area safe.",
#     name="Intrusion_WebSocket_Index",
#     ws_connection_id=connection_id  # Pass WebSocket connection ID
# )

---
## 🎤 Audio Indexing & Transcription

VideoDB RTStream also supports audio analysis for streams with audio content. You can:
- **Index Audio**: Extract structured information from audio streams using AI prompts
- **Start Transcription**: Get real-time speech-to-text transcription

These features work alongside visual indexing to provide comprehensive stream analysis.

In [ ]:
# # Audio Indexing Example (if stream has audio)
# audio_index = intruder_stream.index_audio(
#     prompt="Detect suspicious sounds like glass breaking, door forcing, voices, or footsteps. Extract information about potential intrusion indicators.",
#     batch_config={"type": "time", "value": 30},  # Segment every 30 seconds
#     name="Property_Security_Audio_Index"
# )
#
# # Real-time Transcription Example
# # Get speech-to-text transcription in real-time
# intruder_stream.start_transcript(ws_connection_id=connection_id)
#
# # Poll transcript data
# transcript = intruder_stream.get_transcript(
#     start=0,
#     end=None,
#     page=1,
#     page_size=100
# )
# print("Transcript data:", transcript)
#
# # Stop transcription when done
# intruder_stream.stop_transcript(mode="graceful")

---
## 🚨 Level 1 Breach: Loitering Near the Property

We’ll start by creating an event to detect people **loitering or lingering around the property** without actually touching or interacting with it.

Then, we’ll immediately attach an alert to this event, so you get notified in real time.




---
#### First, let us set up the webhook url we are going to use in the alerts

In [ ]:
# Enter link to your webhook url where you want alerts to go. You can create one simply on pipedream.
webhook_url=""

---

### 📦 Create Event for Loitering Detection

This event will flag when someone is spotted loitering near your property boundary.



In [ ]:
loitering_event_id = conn.create_event(
    event_prompt="Detect if a person is loitering near the house perimeter.",
    label="loitering_near_property"
)
print("Loitering Event ID:", loitering_event_id)


Loitering Event ID: 679ade6411306f35


---

### 📦 Create Alert for Level 1 Breach

As soon as the AI detects a loitering event, it’ll send a webhook alert notification.

In [ ]:
if webhook_url:
  loitering_alert_id = intruder_scene_index.create_alert(
      loitering_event_id,
      callback_url=webhook_url,
      # ws_connection_id=ws.connection_id  # Optional: for WebSocket events
  )
  print("Loitering Alert ID:", loitering_alert_id)
else:
  print("Error: Please provide Webhook URL. Alert cannot be created without it.")

---

## 🚨 Level 2 Breach: Door Checking or Lock Tampering

Next, we’ll set up an event to detect when someone moves close to the front door and appears to check or tamper with the lock — a higher severity intrusion attempt.

Then, we’ll link a dedicated alert to this event.




---

### 📦 Create Event for Intrusion Attempt

This event will flag when a person approaches the property’s door and visibly interacts with it.



In [ ]:
intrusion_event_id = conn.create_event(
    event_prompt="Detect if a person is interacting with the door or visibly checking the lock.",
    label="intrusion_attempt"
)
print("Intrusion Attempt Event ID:", intrusion_event_id)

Intrusion Attempt Event ID: 7bd0ccf7e53637ef


---

### 📦 Create Alert for Level 2 Breach

When AI detects this activity, an instant alert will be routed to your webhook.

In [ ]:
if webhook_url:
  intrusion_alert_id = intruder_scene_index.create_alert(
      intrusion_event_id,
      callback_url=webhook_url,
      # ws_connection_id=ws.connection_id  # Optional: for WebSocket events
  )
  print("Intrusion Alert ID:", intrusion_alert_id)
else:
  print("Error: Please provide Webhook URL. Alert cannot be created without it.")

---

## 🚨 Level 3 Breach: Unauthorized Property Entry

Lastly, we’ll define an event to catch the most serious scenario — **someone entering the property**.

Then, we’ll configure a real-time alert for it.




---

### 📦 Create Event for Property Entry

This event triggers when AI spots someone crossing into the property, opening a door, or entering the house.



In [ ]:
entry_event_id = conn.create_event(
    event_prompt="Detect if a person enters the house or crosses the property boundary unlawfully.",
    label="property_entry"
)
print("Property Entry Event ID:", entry_event_id)


Property Entry Event ID: 796434851654377e


---

### 📦 Create Alert for Level 3 Breach

The AI will send an immediate webhook alert for this highest-severity breach.

In [ ]:
if webhook_url:
  entry_alert_id = intruder_scene_index.create_alert(
      entry_event_id,
      callback_url=webhook_url,
      # ws_connection_id=ws.connection_id  # Optional: for WebSocket events
  )
  print("Property Entry Alert ID:", entry_alert_id)
else:
  print("Error: Please provide Webhook URL. Alert cannot be created without it.")

---
### Lets see the events and alerts we have set up

In [ ]:
def list_rtstream_alerts(rtstream, index_id):
    """
    Prints a list of alerts associated with a given scene index.
    """
    rtstream_scene_index = rtstream.get_scene_index(index_id)
    alerts = rtstream_scene_index.list_alerts()

    for alert in alerts:
        print(f"""🔔 RTStream Alert:
    Alert ID      : {alert['alert_id']}
    Event ID      : {alert['event_id']}
    Label         : {alert['label']}
    Prompt        : {alert['prompt']}
    Status        : {alert['status']}
        """)
        print("-" * 80)

list_rtstream_alerts(intruder_stream, intruder_index_id)


🔔 RTStream Alert:
    Alert ID      : 441c80e43b16fbf5
    Event ID      : 7bd0ccf7e53637ef
    Label         : intrusion_attempt
    Prompt        : Detect if a person is interacting with the door or visibly checking the lock.
    Status        : enabled
        
--------------------------------------------------------------------------------
🔔 RTStream Alert:
    Alert ID      : 5d5bd37c77de2607
    Event ID      : 796434851654377e
    Label         : property_entry
    Prompt        : Detect if a person enters the house or crosses the property boundary unlawfully.
    Status        : enabled
        
--------------------------------------------------------------------------------
🔔 RTStream Alert:
    Alert ID      : c10272cfdf1bb7b1
    Event ID      : 679ade6411306f35
    Label         : loitering_near_property
    Prompt        : Detect if a person is loitering near the house perimeter.
    Status        : enabled
        
---------------------------------------------------------

---

## 📡 Example Alert Payload

A **Level 1: Loitering Around** alert received on the webhook:

```json
{
  "event_id": "event-679ade6411306f35",
  "label": "loitering_near_property",
  "confidence": 0.75,
  "explanation": "A person is detected loitering near the house perimeter, matching the alert context. Confidence is high due to the person's presence in the distance, although the specific activity is not definitively confirmed.",
  "timestamp": "2025-05-27T14:58:34.908641+00:00",
  "start_time": "2025-05-27T20:28:26.061836+05:30",
  "end_time": "2025-05-27T20:28:30.089070+05:30",
  "stream_url": "https://rt.stream.videodb.io/manifests/rts-019710fa-9511-79c3-a924-e229e4815410/1748357906000000-1748357910000000.m3u8"
}
```

✅ Similar alerts will be triggered for Level 2 and Level 3 security breaches.


---
### Let us have a look at the stream_url received in the alert.

In [7]:
alert_stream_url = "https://rt.stream.videodb.io/manifests/rts-019710fa-9511-79c3-a924-e229e4815410/1748480575000000-1748480580000000.m3u8"
video_name = "🚨 Private Property · loitering_near_property"

display_stream(alert_stream_url,video_name)

---

## 🔄 WebSocket Events

As an alternative to webhooks, you can use WebSocket connections to receive real-time events. WebSockets provide a persistent, bidirectional connection that's ideal for interactive applications and dashboards.

**Event Channels:**
- `transcript` - Real-time speech-to-text from `start_transcript()`
- `scene_index` - Visual analysis results from `index_visuals()`
- `audio_index` - Audio analysis results from `index_audio()`
- `alert` - Real-time alert notifications from `create_alert()`

---
-  Let us disable the alerts now.

In [ ]:
intruder_scene_index.disable_alert(loitering_alert_id)
intruder_scene_index.disable_alert(intrusion_alert_id)
intruder_scene_index.disable_alert(entry_alert_id)

- To enable the alert again

In [ ]:
intruder_scene_index.enable_alert(loitering_alert_id)
intruder_scene_index.enable_alert(intrusion_alert_id)
intruder_scene_index.enable_alert(entry_alert_id)

- Now we can stop the stream

In [ ]:
intruder_stream.stop()

---

## 🌙 Wrapping Up: Smarter Property Security Without Stress

In this notebook, we built a **layered AI-powered video surveillance system** to protect your property in real time.

With this in place, property owners can leave home without anxiety, knowing they’ll be immediately notified if anyone is:
- **Loitering nearby**
- **Interacting with the door**
- **Or breaking in**

---

## 🔍 What Else Could We Secure?

This layered security system isn’t limited to homes. It can just as easily protect:
- 🏬 **Retail stores from shoplifters**
- 🏢 **Warehouse perimeters and restricted areas**
- 🛡️ **Military bases and government installations**
- 🏫 **Schools and public buildings**

**What would *you* secure next?**